<img style="float: left;" src="figures/DARTS_21_Sg_grav.PNG" width="20%">   

# <font color='Red'> $\;$ Model for chemical benchmark</font>

First we are going to install open-darts (only need to run it once):

In [ ]:
pip install open-darts

## <font color='blue'>The objectives</font>
In this exercise, we run simulation for a chemical benchmark:
1. Introduce phsyics for super model with multiple inheritance
2. Run model for 1D and 2D reservoir
3. Change parameters of the model and observe changes in the results

## <font color='Blue'>Introduction to super-engine</font>


### Governing equations

For the investigated domain with volume $\Omega$, bounded by surface $\Gamma$, the mass and energy conservation can be expressed in a uniformly integral way, as

\begin{equation}
\frac{\partial}{\partial{t}} \int_{\Omega}{M^c}d{\Omega} + \int_{\Gamma}{{\bf F}^c\bf{\cdot}{\bf{n}}}d{\Gamma} = \int_{\Omega}{Q^c}d{\Omega}.
\end{equation}

Here, $M^c$ denotes the accumulation term for the $c^{\mathrm{th}}$ component ($c = 1, \ldots, n_c$, indexing for the mass components, [e.g., water, $\mathrm{CO_2}$] and $c = n_c + 1$ for the energy quantity); $\bf{F}_c$ refers to the flux term of the $c^{\mathrm{th}}$ component; ${\bf{n}}$ refers to the unit normal pointing outward to the domain boundary;
$Q_c$ denotes the source/sink term of the $c^{\mathrm{th}}$ component.

The mass accumulation term collects each component distribution over $n_p$ fluid phases in a summation form, 

\begin{equation}
    \begin{aligned}
        M^c = \phi\sum\limits^{n_p}_{j=1}x_{cj}\rho_js_j + (1-\phi), \quad c = 1, \ldots, n_c,
    \end{aligned}
\end{equation}

where $\phi$ is porosity, $s_j$ is phase saturation, $\rho_j$ is phase density $[\mathrm{kmol/m^3}]$ and $x_{cj}$ is molar fraction of $c$ component in $j$ phase.

The energy accumulation term contains the internal energy of fluid and rock,

\begin{equation}
    \begin{aligned}
        M^{n_c+1} = \phi\sum\limits^{n_p}_{j=1}\rho_js_jU_j + (1 - \phi)U_r,
    \end{aligned}
\end{equation}

where $U_j$ is phase internal energy $[\mathrm{kJ/kmol}]$ and $U_r$ is rock internal energy $[\mathrm{kJ/m^3}]$.

The rock is assumed compressible and represented by the change of porosity through:

\begin{equation} 
    \phi = \phi_0 \big(1 + c_r (p - p_\mathrm{ref}) \big),
\end{equation}

where $\phi_0$ is the initial porosity, $c_r$ is the rock compressibility [1/bar] and $p_\mathrm{ref}$ is the reference pressure [bars].


The mass flux of each component is represented by the summation over $n_p$ fluid phases,

\begin{equation} 
    \begin{aligned}
        {\bf F}^c = \sum\limits_{j=1}^{n_p}x_{cj}\rho_j {\bf u_j} + s_{j}\rho_{j} \textbf{J}_{cj}, \quad c = 1, \ldots, n_c.
    \end{aligned}
\end{equation}

Here the velocity $\bf{u_j}$ follows the extension of Darcy's law to multiphase flow,

\begin{equation} 
    \small
    {\bf u_j} = \mathbf{K}\frac{k_{rj}}{\mu_j}(\nabla{p_j}-{\bf \gamma_j}\nabla{z}),
\end{equation}

where $\mathbf{K}$ is the permeability tensor $[\mathrm{mD}]$, $k_{rj}$ is the relative permeability of phase $j$, $\mu_j$ is the viscosity of phase $j$ $[\mathrm{mPa\cdot s}]$, $p_j$ is the pressure of phase $j$ [bars], ${\bf \gamma_j}=\rho_j\bf{g}$ is the specific weight $[\mathrm{N/m^3}]$ and $z$ is the depth vector [m].


The $\textbf{J}_{cj}$ is the diffusion flux of component $c$ in phase $j$, which is described by Fick's law as

\begin{equation}
\textbf{J}_{cj} = - \phi \textbf{D}_{cj} \nabla x_{cj},
\label{eq: diffusion equation}
\end{equation}

where $\textbf{D}_{cj}$ is the diffusion coefficient [m$^2$/day].
 

The energy flux includes the thermal convection and conduction terms, \useshortskip

\begin{equation}
    \begin{aligned}
        {\bf F}^{n_c+1} = \sum\limits^{n_p}_{j=1}h_j\rho_j {\bf u_j} + \kappa\nabla{T},
    \end{aligned}
\end{equation}

where $h_j$ is phase enthalpy $[\mathrm{kJ/kmol}]$ and $\kappa$ is effective thermal conductivity $[\mathrm{kJ/m/day/K}]$.


Finally, the source term in mass conservation equations can be present in the following form

\begin{equation}
    \begin{aligned}
        {Q}^{c} = \sum\limits_{k=1}^{n_k}v_{ck}r_k, \quad c = 1, \ldots, n_c,
    \end{aligned}
\end{equation}

where $q_j$ is the phase source/sink term from the well, $v_{ck}$ is the stoichiometric coefficient associated with chemical reaction $k$ for the component $c$ and $r_{k}$ is the rate for the reaction. %Here we assume that equilibrium reactions are absent. 
Similarly, the source term in the energy balance equation can be written as

\begin{equation} 
    \begin{aligned}
        {Q}^{n_c+1} = \sum\limits_{k=1}^{n_k}v_{ek}r_{ek}.
    \end{aligned}
\end{equation}

Here $v_{ek}$ is the stoichiometric coefficient associated with kinetic reaction $k$ for the energy and $r_{ek}$ is the energy rate for kinetic reaction.


The nonlinear equations are discretized with the finite volume method using the multi-point flux approximation on general unstructured mesh in space and with the backward Euler approximation in time. For the $i^{\mathrm{th}}$ reservoir block, the governing equation in discretized residual form reads:

\begin{equation} 
    \begin{aligned}
        R^c_i = V_i \Big(M^{c}_i(\omega_i) - M^{c}_i(\omega^n_{i}) \Big) - 
        \Delta{t} \Big(\sum_l{A_{l}F^{c}_{l}(\omega)} + V_iQ^{c}_{i}(\omega) \Big) = 0, \quad c = 1, \ldots, n_c+1.
    \end{aligned}
\end{equation}

Here $V_i$ is the volume of the $i^{th}$ grid block, $\omega_{i}$ refers to state variables at the current time step, $\omega^{n}_i$ refers to state variables at previous time step, $A_l$ is the contact area between neighboring grids.

### Conservation of mass and energy in operator form 

Pressure, temperature and overall composition are taken as the unified state variables in a given control volume in general-purpose thermal-compositional simulation. Upstream weighting of the physical state is used to determine the flux-related fluid properties determined at the interface $l$. The discretized mass conservation equation in operator form for girdblock (here we omit $i$) reads:

\begin{equation}
V\phi_0[ \alpha_c (\omega) -\alpha_c( \omega_n)]-\Delta t\sum_{l\in L(i)}\sum_{j=1}^{n_p}[\Gamma^l\beta_{cj}^l(\omega^u)\Delta\psi_j^l + \Gamma_d^l\gamma_{j}^l(\omega)\Delta \chi_{cj}]+\Delta t V \delta_c(\omega)=0   \label{eq:operator format}.
\end{equation}

where $V$ is the control volume, $\omega_n$ is the physical state of block $i$ at the previous timestep, $\omega$ is the physical state of block $i$ at the new timestep, $\omega^{u}$ is the physical state of upstream block, $\Gamma^l$ and $\Gamma_d^l$ are the fluid and diffusive transmissibilities respectively and $L(i)$ is a set of interfaces for gridblock $i$.


Here we defined the following state-dependent operators,

\begin{eqnarray}
\alpha_{cf}\left(\omega\right) &=& \Big(1+c_r(p-p_{ref})\Big)\sum_{j=1}^{n_p}x_{cj}\rho_js_j \label{eq:alpha}, \ c = 1,\ldots,n_c;\\
%\alpha_{cr}\left(\omega\right) &=& 0, \ c = 1,\ldots,n_c;\\
\beta_{cj}(\omega) &=& x_{cj}\rho_jk_{rj}/\mu_{j} \label{eq:belta}, \ c = 1,\ldots,n_c, \ j = 1,\ldots,n_p;\\  
\gamma_{j}(\omega) &=& \Big(1+c_r(p-p_{ref})\Big) s_j, \ j = 1,\ldots,n_p;\\  
\chi_{cj}(\omega) &=& D_{cj} \rho_j x_{cj}, \ c = 1,\ldots,n_c, \ j = 1,\ldots,n_p; \\
\delta_{c}(\omega) &=& \sum\limits_{j=1}^{n_p}v_{cj}r_j(\omega),\ c = 1,\ldots,n_c.
\end{eqnarray}

The phase-potential-upwinding (PPU) strategy for OBL parametrization is applied in DARTS to model the gravity and capillary effect. The potential difference of phase $j$ on the interface $l$ between block $1$ and $2$ can be written as:

\begin{equation}
\Delta \psi^l_{j} = p_1-p^c_{j}(\omega_1) - (p_2-p^c_{j}(\omega_2))-\frac{\rho_j(\omega_1)+\rho_j(\omega_2)}{2}g(z_2-z_1), \label{eq:pressure difference}
\end{equation}

where $p^c_{j}$ is the capillary pressure. 




The discretized energy conservation equation in operator form can be written as:

\begin{equation}
\begin{aligned}
V \phi_0 [\alpha_{ef}(\omega) &- \alpha_{ef}(\omega_n) ] - 
\Delta{t}\sum_{l\in L(i)} \sum_{j=1}^{n_p}[\Gamma^l\beta_{ej}^l(\omega^{u})\Delta\psi_{j}^l
+ \Gamma_d^l\gamma_{j}(\omega) \Delta \chi_{ej}]
+ \Delta t V \delta_e(\omega) \\
&+ (1-\phi_0)VU_r [\alpha_{er}(\omega) - \alpha_{er}(\omega_n)] - \Delta{t}\sum\limits_l{ (1-\phi_0)\Gamma_d^l\kappa_r\alpha_{er}(\omega) \Delta \chi_{er}} = 0,
\end{aligned}
\end{equation}

where:

\begin{eqnarray} 
\alpha_{ef}(\omega)&=& \Big(1+c_r(p-p_{ref}) \Big)\sum\limits^{n_p}_{j=1}{\rho_js_jU_j}; 
\label{eq: energy_acc_flux_1} \\
\beta_{ej}(\omega)&=&h_j\rho_j{k_{rj}}/{\mu_j}, \ j = 1,\ldots, n_p;
\label{eq: energy_acc_flux_5}\\
\chi_{ej}(\omega) &=& \kappa_j T_j, \ j = 1,\ldots,n_p;
\label{eq: energy_acc_flux_3} \\
\delta_{e}(\omega) &=& \sum\limits_{j=1}^{n_j}v_{ej}r_{ej}(\omega)
\label{eq: energy_acc_flux_6}
\end{eqnarray}


In addition, for accounting the energy of rock, three additional operators should be defined:

\begin{equation}
\alpha_{er}(\omega)= \frac{1}{1+c_r(p-p_{ref})}, \ 
\label{eq: energy_acc_rock_2} 
\chi_{er}(\omega) = T_r.
\end{equation}

$\alpha_{eri}$ and $\alpha_{erc}$ represent the rock internal energy and rock conduction, respectively. $U_r$ is a state-dependent parameter, thus these two rock energy terms are treated separately.


## <font color='Blue'>Schematics of 1D model</font>

<img style="float: left;" src="figures/chem_bench_1D.png" width="80%">

The first model is a setup with an injection well in the first block and a production well in the last block, no flow boundary conditions from left and right (i.e., $\frac{\partial p}{\partial x}|_{x=0} = \frac{\partial p}{\partial x}|_{x=L} = 0 $).

We use a simplified chemical relationship for this case. It consists of a single chemical reaction (i.e., we cannot reduce the global system of nonlinear equation using the element reduction). The system consists of the following components: {$H_2O$, ${CO_2}$, ${Ca^{+2}}$, ${CO_3^{-2}}$, ${CaCO_3}$} and the kinetic reaction equation consists of

\begin{align}
\label{eq:calcdiss}
    {CaCO_3 <=> Ca^{+2} + CO_3^{-2}}.
\end{align}

Here we assume that the chemical reaction is kinetic. The kinetic rate is written as

\begin{equation}\label{kinetic_react}
    r_k = A_s K_k \left(1 - \frac{Q}{K_{sp}}\right) %x_{CO_2, w}
\end{equation}

where $A_s$ is the reactive surface area, which is a linear function of the solid saturation ($A_s = A_0 \hat{s}_s =(1-\phi_0)\hat{s}_s)$, $K_k$ is the kinetic reaction constant, $Q$ is the activity product and $K_{sp}$  is the equilibrium constant.

## <font color='Blue'>Super-engine physics</font>

Here we introduce custom [Model](https://gitlab.com/open-darts/open-darts-workshop/-/blob/main/model_chem_bench.py) class based on [DartsModel](https://gitlab.com/open-darts/open-darts/-/blob/development/darts-package/darts/models/darts_model.py) which is using [SuperPhysics](https://gitlab.com/open-darts/open-darts/-/tree/development/darts-package/darts/models/physics_sup). 

In [ ]:
from model_chem_bench import Model
from darts.engines import value_vector, redirect_darts_output

redirect_darts_output('run_chemical_1D.log')
n = Model(grid_1D=True)
n.init()
n.params.max_ts = 1e+0

n.run_python(1000)
n.save_restart_data()
n.print_timers()
n.print_stat()


In [ ]:
n.print_and_plot_1D()

## <font color='Blue'>Schematics of 2D model</font>

<img style="float: left;" src="figures/chem_bench_2D.PNG" width="80%">

The second test case consists of a two-dimensional heterogeneous domain. In the model, a zone of high porosity (and permeability) is embedded within a lower porosity (and permeability) zone. The domain extends for 10 [m] in the $y$-direction and all the other measures are mentioned in the figure above. The boundary conditions are constant injection rate on the left with bottom half of the domain pure ${CO_2}$, top half pure ${H_2O}$. The constant pressure is defined on the right boundary (outflow) with no-flow conditions on top and bottom. 